<a href="https://colab.research.google.com/github/LongNguyen1984/Embedded-Machine-Learning-Lab/blob/main/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [70]:
# download the Smarket.csv datafile
df = pd.read_csv('https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Smarket.csv', index_col=1, parse_dates=True)
df.head()

,Unnamed: 0,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,,
2001-01-01,1,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,2,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,3,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,4,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,5,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In this lab, a logistic regression model in order to predict Direction using Lab1 through Lag5 and Volumn. 
We build our model using the glm() function, which is part of formula submodule of (statsmodels)

In [3]:
import statsmodels.formula.api as smf

## We use the formular

'Direction' ~ Lag1 + Lag2 + ... + Lag5 + Volume

In [86]:
formula = 'Direction ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume'

In [87]:
model = smf.glm(formula=formula, data =df, family = sm.families.Binomial())
result = model.fit()
print(result.summary())

                          Generalized Linear Model Regression Results                           
Dep. Variable:     ['Direction[Down]', 'Direction[Up]']   No. Observations:                 1250
Model:                                              GLM   Df Residuals:                     1243
Model Family:                                  Binomial   Df Model:                            6
Link Function:                                    logit   Scale:                          1.0000
Method:                                            IRLS   Log-Likelihood:                -863.79
Date:                                  Wed, 21 Jul 2021   Deviance:                       1727.6
Time:                                          10:18:46   Pearson chi2:                 1.25e+03
No. Iterations:                                       4                                         
Covariance Type:                              nonrobust                                         
                 coef    std e

In [6]:
print("Coefficeients")
print(result.params)
print()
print("p-Values")
print(result.pvalues)
print()
print("Dependent variables")
print(result.model.endog_names)

Coefficeients
Intercept    0.126000
Lag1         0.073074
Lag2         0.042301
Lag3        -0.011085
Lag4        -0.009359
Lag5        -0.010313
Volume      -0.135441
dtype: float64

p-Values
Intercept    0.600700
Lag1         0.145232
Lag2         0.398352
Lag3         0.824334
Lag4         0.851445
Lag5         0.834998
Volume       0.392404
dtype: float64

Dependent variables
['Direction[Down]', 'Direction[Up]']


Predict function will predict the probability that the market will go down, given values of the predictors.

In [7]:
predictions = result.predict()
print(predictions[0:10])

[0.49291587 0.51853212 0.51886117 0.48477764 0.48921884 0.49304354
 0.50734913 0.49077084 0.48238647 0.51116222]


In [8]:
print(np.column_stack((df["Direction"], 
                       result.model.endog)))

[['Up' 0.0]
 ['Up' 0.0]
 ['Down' 1.0]
 ...
 ['Up' 0.0]
 ['Down' 1.0]
 ['Down' 1.0]]


In [9]:
predictions_nomial  = ['Up' if x< 0.5 else 'Down' for x in predictions]


In [10]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df['Direction'], predictions_nomial))

[[145 457]
 [141 507]]


In [17]:
(507 + 145)/(145+457+141+507)

0.5216

In [11]:
print(classification_report(df["Direction"], 
                            predictions_nomial, 
                            digits = 3))

              precision    recall  f1-score   support

        Down      0.507     0.241     0.327       602
          Up      0.526     0.782     0.629       648

    accuracy                          0.522      1250
   macro avg      0.516     0.512     0.478      1250
weighted avg      0.517     0.522     0.483      1250



In [73]:
#df.set_index("Year", inplace = True)
x_train = df.loc[:'2004',:]
y_train = df.loc[:'2004',['Direction']]

x_test = df.loc['2005':,:]
y_test = df.loc['2005':,['Direction']]

In [75]:
x_train.head()

,Unnamed: 0,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
Year,,,,,,,,,
2001-01-01,1,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
2001-01-01,2,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2001-01-01,3,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
2001-01-01,4,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
2001-01-01,5,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [76]:
model = smf.glm(formula, data=x_train, 
                family = sm.families.Binomial())
result = model.fit()

In [77]:
predictions = result.predict(x_test)
predictions_nominal = [ "Up" if x < 0.5 else "Down" for x in predictions]
print(classification_report(y_test, 
                            predictions_nominal, 
                            digits = 3))

              precision    recall  f1-score   support

        Down      0.500     0.315     0.387       111
          Up      0.582     0.752     0.656       141

    accuracy                          0.560       252
   macro avg      0.541     0.534     0.522       252
weighted avg      0.546     0.560     0.538       252



## The better model

In [92]:
formula = 'Direction ~ Lag1 + Lag2'
model = smf.glm(formula, data=x_train, 
                family = sm.families.Binomial()) # Write your code to fit the new model here

result = model.fit()
# -----------------------------------
# This will test your new model; you
# don't need to change anything below
# -----------------------------------result = model.fit()
predictions = result.predict(x_test)
predictions_nominal = [ "Up" if x < 0.5 else "Down" for x in predictions]
print(classification_report(y_test, predictions_nominal, digits = 3))

              precision    recall  f1-score   support

        Down      0.500     0.315     0.387       111
          Up      0.582     0.752     0.656       141

    accuracy                          0.560       252
   macro avg      0.541     0.534     0.522       252
weighted avg      0.546     0.560     0.538       252



In [93]:
print(result.predict(pd.DataFrame([[1.2, 1.1], 
                                   [1.5, -0.8]], 
                                  columns = ["Lag1","Lag2"])))

0    0.520854
1    0.503906
dtype: float64
